In [1]:
import pandas as pd

df = pd.read_excel('인프라 정보.xlsx')
df = df.to_csv("PortInfoA.csv")

In [2]:
import mysql.connector

# MySQL 데이터베이스에 연결 설정
connection = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'project'
)

# 연결 확인 절차
if connection.is_connected():
    print("연결 성공")

연결 성공


In [3]:
df = pd.read_csv('PortInfoA.csv')

print(df.head(5))

   Unnamed: 0 port_code  loc_type info  \
0           0     KRPUS         1  정박지   
1           1     KRPUS         1  정박지   
2           2     KRPUS         1  정박지   
3           3     KRPUS         1  정박지   
4           4     KRPUS         1  정박지   

                                            location  min_ton   max_ton  
0  [{lat:35.07645, lng:129.04538}, {lat:35.07645,...  30000.0  200000.0  
1  [{lat:35.07645, lng:129.02445}, {lat:35.07645,...  30000.0  200000.0  
2  [{lat:35.06336, lng:129.06442}, {lat:35.06336,...  30000.0  200000.0  
3  [{lat:35.04841, lng:129.08572}, {lat:35.04430,...  30000.0  200000.0  
4  [{lat:35.02686, lng:129.04391}, {lat:35.00319,...  30000.0  200000.0  


In [7]:
import numpy as np

# MySQL 커서 생성
cursor = connection.cursor()

# 각 행을 삽입하는 SQL 쿼리 작성
for index, row in df.iterrows():
    sql = """
    INSERT INTO PortInfoA (port_code, loc_type, info, location, min_ton, max_ton)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    loc_type = VALUES(loc_type),
    info = VALUES(info),
    location = VALUES(location),
    min_ton = VALUES(min_ton),
    max_ton = VALUES(max_ton)
    """
    
    # NaN 값을 None으로 변환 (MySQL에서는 None이 NULL로 처리됨)
    values = (
        row['port_code'] if pd.notna(row['port_code']) else None,
        row['loc_type'] if pd.notna(row['loc_type']) else None,
        row['info'] if pd.notna(row['info']) else None,
        row['location'] if pd.notna(row['location']) else None,
        row['min_ton'] if pd.notna(row['min_ton']) else None,
        row['max_ton'] if pd.notna(row['max_ton']) else None
    )
    
    cursor.execute(sql, values)

# 변경 사항을 커밋
connection.commit()

# 연결 해제
cursor.close()
connection.close()

print("CSV 데이터를 MySQL에 성공적으로 삽입했습니다.")

CSV 데이터를 MySQL에 성공적으로 삽입했습니다.
